In [1]:
import csv
import os
from collections import defaultdict
from datetime import datetime
from datetime import timedelta

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
total_colums = [
    # Ejemplar
    'TITULO', 'AUTOR', 'EDITORIAL', 'SUCURSAL', 'DESC_SUCURSAL', 'COD_BARRAS_LIBRO', 'NUM_TITULO', 'SIG_TOPOGRAFICA',
    'DEWEY', 'II_SUMARIO', 'I_SUMARIO', 'EDICION', 'LOCALIZACION_EJEMPLAR', 'DESC_LOCALIZACION_EJEMPLAR', 'POLITICA',
    'DESC_POLITICA_EJEMPLAR', 'ISBN', 'FECHA_REGISTRO_EJEMPLAR', 'FECHA_ADICION_EJEMPLAR', 'FECHA_PUB',
    'ESTADO_EJEMPLAR', 'PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION', 'NIVEL_EDUCACION', 'AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO', 'FECHA_REAL_DEVOLUCION',
]

In [3]:
used_colums = [
    # Ejemplar
    'TITULO', 'AUTOR', 'SUCURSAL', 'COD_BARRAS_LIBRO', 'DEWEY', 'DESC_LOCALIZACION_EJEMPLAR',
    'FECHA_PUB','PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION','NIVEL_EDUCACION','AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO','SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO','FECHA_REAL_DEVOLUCION'
]

# Datos limpios

In [4]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'SUCURSAL': str,
    'COD_BARRAS_LIBRO': str,
    'DEWEY': str,
    'DESC_LOCALIZACION_EJEMPLAR': str,
    'FECHA_PUB': str,
    'PAIS_EJEMPLAR': str,
    'OCUPACION': str,
    'NIVEL_EDUCACION': str,
    'AREA_ESTUDIO': str,
    'SUCURSAL_PRESTAMO': str,
    'SUCURSAL_DEVOLUCION': str,
    'SUCURSAL_DEL_LECTOR': str,
    'COD_BARRAS_LECTOR': str,
    'CATEGORÍA': str,
    'FECHA_PRESTAMO': str,
    'FECHA_REAL_DEVOLUCION': str,
    'DATE_PRESTAMO': pd.tslib.Timestamp,
    'DATE_DEVOLUCION': pd.tslib.Timestamp,
}

prestamos_df = pd.read_csv('prestamos_totales.csv', header=0, dtype=dtypes, parse_dates=['DATE_PRESTAMO', 'DATE_DEVOLUCION'])
prestamos_df = prestamos_df.fillna('')

CPU times: user 23.2 s, sys: 775 ms, total: 24 s
Wall time: 24 s


In [5]:
prestamos_df.shape

(3054360, 20)

In [6]:
prestamos_df.head()

,TITULO,AUTOR,SUCURSAL,COD_BARRAS_LIBRO,DEWEY,DESC_LOCALIZACION_EJEMPLAR,FECHA_PUB,PAIS_EJEMPLAR,OCUPACION,NIVEL_EDUCACION,AREA_ESTUDIO,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION,SUCURSAL_DEL_LECTOR,COD_BARRAS_LECTOR,CATEGORÍA,FECHA_PRESTAMO,FECHA_REAL_DEVOLUCION,DATE_PRESTAMO,DATE_DEVOLUCION
0,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429527,CTA,22/01/2015,04/02/2015,2015-01-22,2015-02-04
1,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88416381,CTA,31/03/2015,07/04/2015,2015-03-31,2015-04-07
2,Quisiera tener ...,"Zoboli, Giovanna.",ARM,2284458,853,Infantil,2010,mx,,,,ARM,ARM,ARM,88381353,CTA,07/11/2015,17/11/2015,2015-11-07,2015-11-17
3,Johannes Gutenberg,"Borràs Perelló, Lluís.",ARM,2288922,925,Infantil,2010,sp,Empleado,Técnico,Adm. de empresas,ARM,ARM,ARM,88007424,CEM,16/01/2015,16/01/2015,2015-01-16,2015-01-16
4,La fantástica leyenda de: la princesa y el dragón,"Alins, Sonia.",ARM,2288896,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429529,CTA,21/01/2015,28/01/2015,2015-01-21,2015-01-28


# Libros viajeros

In [7]:
sucursales = {
    'ARM': {'name': 'Armenia', 'latitude': 4.53, 'longitude': -75.68},
    'BAR': {'name': 'Barranquilla', 'latitude': 10.963889, 'longitude': -74.796389},
    'BLAA': {'name': 'Luis Angel Arango', 'latitude': 4.59677, 'longitude': -74.072848},
    'BUC': {'name': 'Bucaramanga', 'latitude': 7.133333, 'longitude': -73},
    'BUE': {'name': 'Buenaventura', 'latitude': 3.877222, 'longitude': -77.026667},
    'CAL': {'name': 'Cali', 'latitude': 3.420556, 'longitude': -76.522222},
    'CAR': {'name': 'Cartagena', 'latitude': 10.4, 'longitude': -75.5},
    'CASAGO': {'name': 'Casa Gómez Campuzano', 'latitude': 4.6628468, 'longitude': -74.0512116},
    'CATMAN': {'name': 'Catalogación y Mantenimiento de Colecciones', 'latitude': 4.5769749, 'longitude': -74.0880581},
    'CUC': {'name': 'Cúcuta', 'latitude': 7.894167, 'longitude': -72.503889},
    'ESPE': {'name': 'Especial material audiovisual', 'latitude': 4.59677, 'longitude': -74.072848},
    'FLO': {'name': 'Florencia', 'latitude': 1.614167, 'longitude': -75.611667},
    'GIR': {'name': 'Girardot', 'latitude': 4.305, 'longitude': -74.801667},
    'HON': {'name': 'Honda', 'latitude': 5.204167, 'longitude': -74.741667},
    'IBA': {'name': 'Ibagué', 'latitude': 4.433333, 'longitude': -75.233333},
    'IEC': {'name': 'Investigaciones Económicas', 'latitude': 4.5769749, 'longitude': -74.0880581},
    'IPI': {'name': 'Ipiales', 'latitude': 0.830278, 'longitude': -77.644444},
    'LET': {'name': 'Leticia', 'latitude': -4.205278, 'longitude': -69.932778},
    'MAN': {'name': 'Manizales', 'latitude': 5.1, 'longitude': -75.55},
    'MED': {'name': 'Medellín', 'latitude': 6.230833, 'longitude': -75.590556},
    'MON': {'name': 'Montería', 'latitude': 8.75, 'longitude': -75.883333},
    'NEI': {'name': 'Neiva', 'latitude': 2.998611, 'longitude': -75.304444},
    'PAS': {'name': 'Pasto', 'latitude': 1.207778, 'longitude': -77.277222},
    'PER': {'name': 'Pereira', 'latitude': 4.814278, 'longitude': -75.694558},
    'POP': {'name': 'Popayán', 'latitude': 2.454167, 'longitude': -76.609167},
    'QUI': {'name': 'Quibdó', 'latitude': 5.692222, 'longitude': -76.658056},
    'RIO': {'name': 'Riohacha', 'latitude': 11.544167, 'longitude': -72.906944},
    'SAN': {'name': 'San Andrés', 'latitude': 12.583333, 'longitude': -81.7},
    'SIN': {'name': 'Sincelejo', 'latitude': 9.295, 'longitude': -75.396111},
    'STM': {'name': 'Santa Marta', 'latitude': 11.241944, 'longitude': -74.205278},
    'TUN': {'name': 'Tunja', 'latitude': 5.533333, 'longitude': -73.366667},
    'VAL': {'name': 'Valledupar', 'latitude': 10.483333, 'longitude': -73.25},
    'VIL': {'name': 'Villavicencio', 'latitude': 4.15, 'longitude': -73.633333},
}

In [8]:
sucursales_bogota_no_blaa = ['CASAGO', 'IEC', 'ESPE', 'CATMAN']

In [9]:
sucursales_bogota = sucursales_bogota_no_blaa + ['BLAA']

## Modelos de viaje

In [10]:
# MODEL   SUC_LIBRO  SUC_PRéST  SUC_DEVOLUCIóN    %
#  mABC      A          B          C             0.23%
#  mBBC      B          B          C             2.05%
#  mABB      A          B          B            13.61%
#  mABA      A          B          A             2.20%
#  mBBB      B          B          B            81.91%

#### Modelo ABC

In [11]:
prestamos_mABC_df = prestamos_df[
    (prestamos_df['SUCURSAL'] != prestamos_df['SUCURSAL_PRESTAMO']) &
    (prestamos_df['SUCURSAL_PRESTAMO'] != prestamos_df['SUCURSAL_DEVOLUCION']) &
    (prestamos_df['SUCURSAL_DEVOLUCION'] != prestamos_df['SUCURSAL'])
]

In [12]:
len(prestamos_mABC_df)

7063

In [13]:
per = 100 * len(prestamos_mABC_df) / len(prestamos_df)
'%.2f%%' % per

'0.23%'

In [14]:
prestamos_mABC_df.groupby(['SUCURSAL', 'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION'])[['TITULO']] \
    .count() \
    .sort_values('TITULO', ascending=False) \
    .head(20)

TITULO
SUCURSAL SUCURSAL_PRESTAMO SUCURSAL_DEVOLUCION        
BLAA     ESPE              CASAGO                 1783
CATMAN   ESPE              BLAA                   1081
         BLAA              CASAGO                  181
TUN      CASAGO            BLAA                    164
CAR      CASAGO            BLAA                    113
TUN      BLAA              CASAGO                   93
IBA      CASAGO            BLAA                     90
GIR      CASAGO            BLAA                     80
BLAA     MAN               PER                      79
         PER               MAN                      76
PER      CASAGO            BLAA                     71
MAN      CASAGO            BLAA                     69
CATMAN   CASAGO            BLAA                     64
CAR      BLAA              CASAGO                   59
BLAA     IPI               PAS                      51
PER      ESPE              BLAA                     49
PAS      CASAGO            BLAA                     48
BLAA     CUC               BUC                      47
IBA      BLAA              CASAGO                   45
FLO      CASAGO            BLAA                     41

#### Modelo BBC

In [15]:
prestamos_mBBC_df = prestamos_df[
    (prestamos_df['SUCURSAL'] == prestamos_df['SUCURSAL_PRESTAMO']) &
    (prestamos_df['SUCURSAL_PRESTAMO'] != prestamos_df['SUCURSAL_DEVOLUCION']) &
    (prestamos_df['SUCURSAL_DEVOLUCION'] != prestamos_df['SUCURSAL'])
]

In [16]:
len(prestamos_mBBC_df)

62598

In [17]:
per = 100 * len(prestamos_mBBC_df) / len(prestamos_df)
'%.2f%%' % per

'2.05%'

In [18]:
prestamos_mBBC_df.groupby(['SUCURSAL', 'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION'])[['TITULO']] \
    .count() \
    .sort_values('TITULO', ascending=False) \
    .head(20)

TITULO
SUCURSAL SUCURSAL_PRESTAMO SUCURSAL_DEVOLUCION        
BLAA     BLAA              CASAGO                49364
                           TUN                    1127
                           CAL                    1071
                           VIL                     802
                           MED                     604
TUN      TUN               BLAA                    486
BLAA     BLAA              BUC                     465
                           IBA                     416
CASAGO   CASAGO            BLAA                    402
BLAA     BLAA              NEI                     301
                           CAR                     273
                           PAS                     262
                           BAR                     260
MAN      MAN               PER                     235
BLAA     BLAA              POP                     233
                           GIR                     230
PAS      PAS               IPI                     222
IBA      IBA               BLAA                    214
PER      PER               MAN                     211
BLAA     BLAA              MAN                     207

#### Modelo ABB

In [19]:
prestamos_mABB_df = prestamos_df[
    (prestamos_df['SUCURSAL'] != prestamos_df['SUCURSAL_PRESTAMO']) &
    (prestamos_df['SUCURSAL_PRESTAMO'] == prestamos_df['SUCURSAL_DEVOLUCION']) &
    (prestamos_df['SUCURSAL_DEVOLUCION'] != prestamos_df['SUCURSAL'])
]

In [20]:
len(prestamos_mABB_df)

415679

In [21]:
per = 100 * len(prestamos_mABB_df) / len(prestamos_df)
'%.2f%%' % per

'13.61%'

In [22]:
prestamos_mABB_df.groupby(['SUCURSAL', 'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION'])[['TITULO']] \
    .count() \
    .sort_values('TITULO', ascending=False) \
    .head(20)

TITULO
SUCURSAL SUCURSAL_PRESTAMO SUCURSAL_DEVOLUCION        
BLAA     CASAGO            CASAGO                71900
         BUC               BUC                   36042
         CAL               CAL                   22855
         MAN               MAN                   18004
         TUN               TUN                   17788
         POP               POP                   16354
         PER               PER                   15991
         PAS               PAS                   14689
         VIL               VIL                   14473
         MED               MED                   13578
         NEI               NEI                   11334
         BAR               BAR                   11218
         IBA               IBA                   10510
         ARM               ARM                    9235
         CAR               CAR                    6764
         FLO               FLO                    5523
CATMAN   BLAA              BLAA                   5486
BLAA     VAL               VAL                    5465
         CUC               CUC                    5065
         MON               MON                    4855

#### Modelo ABA

In [23]:
prestamos_mABA_df = prestamos_df[
    (prestamos_df['SUCURSAL'] != prestamos_df['SUCURSAL_PRESTAMO']) &
    (prestamos_df['SUCURSAL_PRESTAMO'] != prestamos_df['SUCURSAL_DEVOLUCION']) &
    (prestamos_df['SUCURSAL_DEVOLUCION'] == prestamos_df['SUCURSAL'])
]

In [24]:
len(prestamos_mABA_df)

67289

In [25]:
per = 100 * len(prestamos_mABA_df) / len(prestamos_df)
'%.2f%%' % per

'2.20%'

In [26]:
prestamos_mABA_df.groupby(['SUCURSAL', 'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION'])[['TITULO']] \
    .count() \
    .sort_values('TITULO', ascending=False) \
    .head(20)

TITULO
SUCURSAL SUCURSAL_PRESTAMO SUCURSAL_DEVOLUCION        
BLAA     ESPE              BLAA                  51123
         CASAGO            BLAA                  13665
         TUN               BLAA                    236
         VIL               BLAA                    225
CASAGO   BLAA              CASAGO                  186
BLAA     CAL               BLAA                    149
         BUC               BLAA                    143
         MED               BLAA                    112
         IBA               BLAA                    108
         ARM               BLAA                     99
         NEI               BLAA                     96
         PER               BLAA                     66
         BAR               BLAA                     56
         POP               BLAA                     55
         FLO               BLAA                     52
         CATMAN            BLAA                     52
         MAN               BLAA                     48
         GIR               BLAA                     45
TUN      BLAA              TUN                      44
BLAA     CUC               BLAA                     39

#### Modelo BBB

In [27]:
prestamos_mBBB_df = prestamos_df[
    (prestamos_df['SUCURSAL'] == prestamos_df['SUCURSAL_PRESTAMO']) &
    (prestamos_df['SUCURSAL_PRESTAMO'] == prestamos_df['SUCURSAL_DEVOLUCION']) &
    (prestamos_df['SUCURSAL_DEVOLUCION'] == prestamos_df['SUCURSAL'])
]

In [28]:
len(prestamos_mBBB_df)

2501731

In [29]:
per = 100 * len(prestamos_mBBB_df) / len(prestamos_df)
'%.2f%%' % per

'81.91%'

In [30]:
prestamos_mBBB_df.groupby(['SUCURSAL', 'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION'])[['TITULO']] \
    .count() \
    .sort_values('TITULO', ascending=False)

,,,TITULO
SUCURSAL,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION,
BLAA,BLAA,BLAA,1448903
TUN,TUN,TUN,198423
PAS,PAS,PAS,120794
POP,POP,POP,120373
IBA,IBA,IBA,97007
PER,PER,PER,77466
MAN,MAN,MAN,71541
IPI,IPI,IPI,63001
GIR,GIR,GIR,34695


In [31]:
prestamos_mBBC_df.groupby(['SUCURSAL', 'SUCURSAL_PRESTAMO'])[['TITULO']] \
    .count() \
    .sort_values('TITULO', ascending=False)

,,TITULO
SUCURSAL,SUCURSAL_PRESTAMO,
BLAA,BLAA,56873
TUN,TUN,701
PER,PER,518
MAN,MAN,510
IBA,IBA,488
PAS,PAS,481
CASAGO,CASAGO,426
POP,POP,341
IPI,IPI,327


In [32]:
indices = set(prestamos_mBBB_df.groupby('SUCURSAL').count().index) \
          .union(prestamos_mBBC_df.groupby(['SUCURSAL', 'SUCURSAL_PRESTAMO']).count().index.get_level_values(0))
for s in sucursales:
    if s not in indices:
        print(s)

BAR
VIL
BUC
ESPE


#### Total

In [33]:
len(prestamos_mABC_df) + len(prestamos_mBBC_df) + len(prestamos_mABB_df) + len(prestamos_mABA_df) + len(prestamos_mBBB_df)

3054360

In [34]:
len(prestamos_df)

3054360

#### % préstamos sin viaje

In [35]:
per = 100 * len(prestamos_mBBB_df) / len(prestamos_df)
'%.2f%%' % per

'81.91%'

#### % préstamos con viaje

In [36]:
per = 100 * (len(prestamos_mABC_df) + len(prestamos_mBBC_df) + len(prestamos_mABB_df) + len(prestamos_mABA_df)) / len(prestamos_df)
'%.2f%%' % per

'18.09%'